#### Dashboard:

In [ ]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_auth
import json

# Carregar dados de autenticação de um arquivo JSON:
with open(r'C:\Users\diego\OneDrive\Área de Trabalho\Programação\Hashtag Programação\Material Aulas Python Impressionador\autenticacao_dash.json', 'r') as arquivo_json:
    dados_autenticacao = json.load(arquivo_json)

# Acessar a variável USUARIOS dos dados de autenticação:
USUARIOS = dados_autenticacao['USUARIOS']

#Criando o aplicativo Dash:
app = Dash(__name__)

#Solicitando a autenticação:
auth = dash_auth.BasicAuth(app, USUARIOS)

#Importando o banco de dados:
df = pd.read_excel(r'Arquivo Mentoria Dash\Vendas.xlsx')

#Criando os gráficos:
fig = px.bar(df, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
fig2 = px.scatter(df, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)

#Criando lista de marcas:
lista_marcas = list(df['Marca'].unique())
lista_marcas.append('Todas')

#Criando lista países:
lista_paises = list(df['País'].unique())
lista_paises.append('Todos')

# Criando o Layout da página do Dash:
app.layout = html.Div(children=[
    html.H1(children='Meu Dashboard'),

    html.H2(children='Dashboard de Vendas em Python'),
    
    html.H3(children='Vendas de cada Produto por Loja', id='subtitulo'),
    
    html.Div(children=[
        dcc.Dropdown(options=lista_paises, value='Todos', id='selecao_pais'),
    ], style={"width": "50%", "margin": "auto"}),
    
    dcc.RadioItems(options=lista_marcas, value='Todas', id='selecao_marcas'),
    
    dcc.Graph(id='vendas_por_loja'),
    dcc.Graph(id='distribuicao_vendas'),
], style={'text-align': 'center'})

#Callbacks:

#Definir qual país será filtrado:
@app.callback(
    Output('selecao_pais', 'options'),
    Input('selecao_marcas', 'value'),
)
def opcoes_pais(marca):
    if marca == 'Todas':
        nova_lista_paises = list(df['País'].unique())
        nova_lista_paises.append('Todos')
    else:
        df_filtrada = df.loc[df['Marca']==marca, :]
        nova_lista_paises = list(df_filtrada['País'].unique())
        nova_lista_paises.append('Todos')
    return nova_lista_paises

#Filtrando os Gráficos:
@app.callback(
    Output('subtitulo', 'children'),
    Output('vendas_por_loja', 'figure'),
    Output('distribuicao_vendas', 'figure'),
    Input('selecao_marcas', 'value'),
    Input('selecao_pais', 'value'),
)
def selecionar_marca(marca, pais):
    if marca == 'Todas' and pais == 'Todos':
        texto = 'Vendas de cada Produto por Loja'
        fig = px.bar(df, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
        fig2 = px.scatter(df, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)
    else:
        df_filtrada = df
        if marca != 'Todas':
            df_filtrada = df_filtrada.loc[df_filtrada['Marca']==marca, :]
        if pais != 'Todos':
            df_filtrada = df_filtrada.loc[df_filtrada['País']==pais, :]
        
        texto = f'Vendas de cada Produto por Loja da Marca {marca} e do País {pais}'
        fig = px.bar(df_filtrada, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
        fig2 = px.scatter(df_filtrada, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)
    return texto, fig, fig2

# Colocando o site(dashboard) no ar:
if __name__ == '__main__':
    app.run_server(debug=False)